In [ ]:
!pip install keras4torch
!pip install einops


import os
import keras4torch
from   keras4torch.callbacks  import ModelCheckpoint,LRScheduler
import torch
import torch.nn    as nn
import torch.optim as optim
import  torch.nn.functional as     F
import numpy       as np
import pandas      as pd
from copy import deepcopy

In [ ]:
import  matplotlib.pyplot   as     plt
from    sklearn.preprocessing import StandardScaler, QuantileTransformer
from    datetime import datetime
import  gc
import STab
from STab import MyRegreLoss,CatMap,Num_Cat
from   STab import mainmodel, LWTA, Gsoftmax
MainModel=mainmodel.MainModel

In [ ]:
##Split the data 
X_test=pd.DataFrame(np.load('Data/house/X_num_test.npy')).astype(np.float64).values
y_test=pd.DataFrame(np.load('Data/house/Y_test.npy')).astype(np.float64)

X_train=pd.DataFrame(np.load('Data/house/X_num_train.npy')).astype(np.float64).values
Y_train=pd.DataFrame(np.load('Data/house/Y_train.npy')).astype(np.float64)


X_valid=pd.DataFrame(np.load('Data/house/X_num_val.npy')).astype(np.float64).values
y_valid=pd.DataFrame(np.load('Data/house/Y_val.npy')).astype(np.float64)

In [ ]:
#Normalise In
scalerX = QuantileTransformer(output_distribution='normal')

scalerX.fit(X_train)

X_train = scalerX.transform(X_train).astype(np.float64)
X_valid = scalerX.transform(X_valid).astype(np.float64)
X_test  = scalerX.transform(X_test).astype(np.float64)

In [ ]:
scale = 10000
bias  = 3.33

y_valid = y_valid/scale-bias
y_test  = y_test/scale-bias
Y_train = Y_train/scale-bias

In [ ]:
def True_MSE(p,t):
        return np.mean( np.square(p-t))

In [ ]:
checkpoint='saved/savefileHO'

In [ ]:


Or_model = MainModel(
    categories        = (),
    num_continuous    = 16,
    dim               = 128  ,
    dim_out           = 1,
    depth             = 4,
    heads             = 8,
    attn_dropout      = 0.125 ,
    ff_dropout        = 0.125,
    U                 = 2,
    cases             = 1

)

no_model = Num_Cat(Or_model,num_number=16,Sample_size=16)
model    = keras4torch.Model(no_model,).build([16])
Or_model.numerical_embedder.lwta.temperature=0.69
#Warm Up
optimizer=torch.optim.AdamW(model.parameters(),lr=0.0001,weight_decay=0.0001,)
sch=torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.001, total_iters=10,  verbose=False)
model.compile(optimizer=optimizer, loss=MyRegreLoss(0.01,1), metrics=['mse'])
callbacks=[LRScheduler(sch)]
model.fit(X_train, Y_train.values,
                                  epochs=10, batch_size=256,
                                  validation_data=(X_valid,y_valid.values),
                                  verbose=2,validation_batch_size=512,
                                  callbacks=callbacks)

#Main Train
optimizer=torch.optim.AdamW(model.parameters(),lr=0.001,weight_decay=0.0001,)
model.compile(optimizer=optimizer, loss=MyRegreLoss(0.01,1), metrics=['mse'])
scheduler =LRScheduler( torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5,min_lr=0.00001))
callbacks=[ModelCheckpoint(checkpoint,monitor='val_mse',mode='min'),scheduler]
model.fit(X_train, Y_train.values,
        epochs=115, batch_size=256,
        validation_data=(X_valid,y_valid.values),
        verbose=2,validation_batch_size=512,
        callbacks=callbacks)

model.load_weights(checkpoint)


no_model.reset_Sample_size(1)
logits=pd.DataFrame()

for i in range(0,64):

        logits[i]=pd.DataFrame(model.predict(X_test,batch_size=4096))


Test = (True_MSE(logits.mean(axis=1).values.reshape((-1)),y_test.values.reshape((-1))))
print('Test Score:',Test)
#Test Score: 9.699489410492822
